In [1]:
suppressMessages(library(Seurat))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(viridis))
suppressMessages(library(ggplot2))
options(warn=-1)

In [2]:
resolutions <- seq(0.2, 1, 0.1)

In [3]:
# obj <- readRDS(glue::glue("../../../visiumSpatial/from_rico/processed_visium/objects/AKK004_157772.rds"))

# obj

# all.markers <- readRDS(glue::glue("../data/coembed/MarkerGenes/res.0.2.rds"))
# gene.use <- intersect(rownames(obj), rownames(all.markers))
# all.markers <- all.markers[gene.use, ]

# all.markers <- all.markers %>%
#     subset(p_val_adj < 0.05) %>%
#     group_by(cluster) %>%
#     top_n(n = 50, wt = avg_log2FC) %>%
#     subset(., select = c("cluster", "gene"))

# all.markers <- split(all.markers, all.markers$cluster)

# for(cl in names(all.markers)){
#    all.markers[[cl]] <- all.markers[[cl]]$gene 
# }

# DefaultAssay(obj) <- "c2l_props"
# p1 <- SpatialFeaturePlot(obj, features = "Fib") +
# scale_fill_viridis(option = "D")

# DefaultAssay(obj) <- "c2l"
# p2 <- SpatialFeaturePlot(obj, features = "Fib") +
# scale_fill_viridis(option = "D")

# options(repr.plot.height = 5, repr.plot.width = 10)

# p1 + p2

# fib_score <- obj@assays$c2l_props@data["Fib", ]

# DefaultAssay(obj) <- "SCT"
# obj <- AddModuleScore(obj, features = all.markers)

# ## multiply the module score with cell2location deconvolution score
# # get module score
# df_module_score <- as.data.frame(obj@meta.data)%>%
#     select(., contains("Cluster"))

# df_module_score <- df_module_score * fib_score

# obj <- AddMetaData(obj, metadata = df_module_score)

# min_score <- min(df_module_score)
# max_score <- max(df_module_score)

# plotlist <- SpatialFeaturePlot(obj, features = c("Cluster1", "Cluster2", "Cluster3", "Cluster4", "Cluster5"), combine = FALSE)

# plotlist <- lapply(plotlist, function(p){
#     p <- p + scale_fill_viridis(limits = c(min_score, max_score))
# })

# p <- patchwork::wrap_plots(plotlist, nrow = 2)

# options(repr.plot.height = 10, repr.plot.width = 20)

# p

In [4]:
df_anno <- read.csv("../../../visiumSpatial/metadata/sample_annotation.csv", header = TRUE)

patient_region_id <- df_anno$patient_region_id
patient_region_id <- stringr::str_replace_all(patient_region_id, "/", "_")

names(patient_region_id) <- df_anno$sample_id

In [ ]:
for(res in resolutions){
    if(!dir.exists(glue::glue("../data/visium/cell_states_mapping/res_{res}"))){
        dir.create(glue::glue("../data/visium/cell_states_mapping/res_{res}"), recursive = TRUE)
    }
    
    for(sample in names(patient_region_id)){
        message(glue::glue("processing sample: {sample}"))

        # load visium object
        obj <- readRDS(glue::glue("../../../visiumSpatial/from_rico/processed_visium/objects/{sample}.rds"))
    
        DefaultAssay(obj) <- "c2l_props"
        p1 <- SpatialFeaturePlot(obj, features = "Fib", max.cutoff = "q99.5") + ggtitle("c2l: proportion") +
            scale_fill_viridis()
   
        DefaultAssay(obj) <- "c2l"
        p2 <- SpatialFeaturePlot(obj, features = "Fib", max.cutoff = "q99.5") + ggtitle("c2l: count")+
            scale_fill_viridis()

        DefaultAssay(obj) <- "SCT"
        
        # load marker genes
        all.markers <- readRDS(glue::glue("../data/coembed/MarkerGenes/res.{res}.rds"))
        
        # only keep genes that are found in spatial dta
        gene.use <- intersect(rownames(obj), rownames(all.markers))
        all.markers <- all.markers[gene.use, ]
        
        # get number of clusters
        n_clusters <- length(unique(all.markers$cluster))
        
        # get the marker genes for each cluster
        all.markers <- all.markers %>%
            subset(p_val_adj < 0.05) %>%
            subset(., select = c("cluster", "gene"))
        
        # convert dataframe to list for module score calculation
        all.markers <- split(all.markers, all.markers$cluster)
        for(cl in names(all.markers)){
           all.markers[[cl]] <- all.markers[[cl]]$gene 
        }
        
        # get module score
        obj <- AddModuleScore(obj, features = all.markers, assay = "SCT")
        
        df_module_score <- as.data.frame(obj@meta.data)%>%
            select(., contains("Cluster"))


        # plot module score after normalization by proportion from cell2location
        fib_score_prop <- obj@assays$c2l_props@data["Fib", ]
        df_module_score_prop <- df_module_score * fib_score_prop
        df_module_score_prop[df_module_score_prop <0] <- 0
        
        obj <- AddMetaData(obj, metadata = df_module_score_prop)
        min_score <- min(df_module_score_prop)
        max_score <- max(df_module_score_prop)
        features <- paste0("Cluster", 1:n_clusters)
        
        plotlist <- SpatialFeaturePlot(obj, features = features, combine = FALSE, max.cutoff = "q99")
        plotlist <- lapply(1:length(plotlist), function(i){
            cl <- i - 1
           p <- plotlist[[i]] + ggtitle(glue::glue("Cluster{cl}")) + theme(legend.title = element_blank()) +
            scale_fill_viridis()
           return(p)
        })
        ps1 <- patchwork::wrap_plots(plotlist, nrow = 2) +
            patchwork::plot_annotation(title = "Normalized by proportion")

        # plot module score after normalization by count from cell2location
        fib_score_count <- obj@assays$c2l@data["Fib", ]
        df_module_score_count <- df_module_score * fib_score_count
        df_module_score_count[df_module_score_count <0] <- 0
        
        obj <- AddMetaData(obj, metadata = df_module_score_count)
        min_score <- min(df_module_score_count)
        max_score <- max(df_module_score_count)
        features <- paste0("Cluster", 1:n_clusters)
        
        plotlist <- SpatialFeaturePlot(obj, features = features, combine = FALSE, max.cutoff = "q99")
        plotlist <- lapply(1:length(plotlist), function(i){
            cl <- i - 1
           p <- plotlist[[i]] + ggtitle(glue::glue("Cluster{cl}")) + theme(legend.title = element_blank()) +
            scale_fill_viridis()
           return(p)
        })
        ps2 <- patchwork::wrap_plots(plotlist, nrow = 2) +
            patchwork::plot_annotation(title = "Normalized by count")
        
        
        patient_region <- patient_region_id[sample]

        pdf(glue::glue("../data/visium/cell_states_mapping/res_{res}/{sample}_{patient_region}.pdf"), width = 20, height = 10)
        print(p1+p2)
        print(ps1)
        print(ps2)
        dev.off()
    }
}

processing sample: AKK006_157771

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

processing sample: Visium_1_CK279

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

processing sample: Visium_3_CK281

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

processing sample: Visium_4_CK282

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace th

In [ ]:
sessionInfo()